# RAG System with Gemini and FAISS
# ================================
This is an initial version of rag that will be further improved with more modules, logging etc.

In [2]:
# Import necessary libraries
import os
from typing import List, Union
from dotenv import load_dotenv
import requests
from langchain.schema import Document
from langchain.prompts import PromptTemplate
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_google_genai import GoogleGenerativeAI
from langchain.chains import RetrievalQA

## 1. Configuration
First, let's set up the configuration for our RAG system.

In [3]:
# Load environment variables
load_dotenv()

# Configuration settings
EMBEDDING_MODEL = "text-embedding-ada-002"
CHUNK_SIZE = 800
CHUNK_OVERLAP = 80
RETRIEVER_K = 4
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")

if not GEMINI_API_KEY:
    raise ValueError("GEMINI_API_KEY not found in environment variables")

# Initialize LLM
llm = GoogleGenerativeAI(
    api_key=GEMINI_API_KEY,
    model="gemini-2.0-flash"
)

## 2. Document Processing Functions
Let's define functions to download and process PDF documents.

In [4]:
def download_pdf(url: str, folder: str = 'documents') -> str:
    """
    Downloads PDF from given URL
    
    Args:
        url (str): URL to download from
        folder (str): Destination folder
        
    Returns:
        str: Path to downloaded file
    """
    os.makedirs(folder, exist_ok=True)
    filename = os.path.basename(url)
    filepath = os.path.join(folder, filename)
    
    response = requests.get(url, timeout=10)
    response.raise_for_status()
    
    with open(filepath, 'wb') as f:
        f.write(response.content)
    return filepath

def process_document(documents: Union[str, List[str]], chunk_size: int, chunk_overlap: int) -> List[Document]:
    """
    Process document from URL to chunks

    Args:
        ocuments (Union[str, List[str]]): A single URL or list of document URLs
        chunk_size (int): Size of each text chunk
        chunk_overlap (int): Overlap between chunks

    Returns:
        List[Document]: List of document chunks
    """
    loaded_docs = []

    if isinstance(documents, str):
        documents = [documents]
    elif not isinstance(documents, list):
        print("Error: 'documents' must be a string or a list of strings.")
        return []
    
    if not documents:
        print("Warning: No document URLs provided.")
        return []

    for url in documents:
        try:
            pdf_path = download_pdf(url)
            pdf_docs = PyPDFLoader(pdf_path).load()
            loaded_docs.extend(pdf_docs)
        except Exception as e:
            print(f"Failed to process document from URL '{url}': {e}")
            continue

    if not loaded_docs:
        print("Warning: No documents were successfully loaded.")
        return []

    try:
        splitter = RecursiveCharacterTextSplitter(
            chunk_size=chunk_size,
            chunk_overlap=chunk_overlap
        )
        chunks = splitter.split_documents(loaded_docs)
        return chunks
    except Exception as e:
        print(f"Error during document splitting: {e}")
        return []

## 3. Vector Store Creation
Now let's create a function to build our vector store from documents.

In [5]:
def create_vectorstore(documents: List[Document], embedding_model: str) -> FAISS:
    """
    Create and save a FAISS vector store from documents
    
    Args:
        documents (List[Document]): List of document chunks
        embedding_model (str): Name of the embedding model
        
    Returns:
        FAISS: Vector store for document retrieval
    """
    embeddings = OpenAIEmbeddings(model=embedding_model)
    
    vectorstore = FAISS.from_documents(
        documents=documents,
        embedding=embeddings
    )
    
    # Save vector store for later use
    vectorstore.save_local("faiss_index")
    
    return vectorstore

## 4. RAG Chain Creation
Let's create our RAG chain with the Gemini model.

In [7]:
def create_rag_chain(vectorstore: FAISS, llm, retriever_k: int = 4) -> RetrievalQA:
    """
    Create a conversational retrieval chain
    
    Args:
        vectorstore (FAISS): Vector store for retrieval
        llm: Language model
        retriever_k (int): Number of documents to retrieve
        
    Returns:
        RetrievalQA: RAG chain
    """
    template = """Use the following pieces of context to answer the question. Explain like you are talking to a 5-year-old. If the question is not related to the context, say "I don't know".
    If you don't know the answer, just say that you don't know, don't try to make up an answer.

    Context: {context}

    Question: {question}

    Provide a clear and concise answer. If possible, cite specific parts from the context.

    Answer:"""

    prompt = PromptTemplate(
        template=template,
        input_variables=["context", "question"]
    )

    # Create the chain
    chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=vectorstore.as_retriever(
            search_type="similarity",
            search_kwargs={"k": retriever_k}
        ),
        chain_type_kwargs={"prompt": prompt},
        return_source_documents=True
    )
    return chain

## 5. Building the Complete RAG System
Now let's put everything together to build our RAG system.

In [8]:
def build_rag_system(document_url: str):
    """
    Build the complete RAG system from a document URL
    
    Args:
        document_url (str): URL of the document
        
    Returns:
        ConversationalRetrievalChain: Ready-to-use RAG chain
    """
    # Process document
    chunks = process_document(document_url, CHUNK_SIZE, CHUNK_OVERLAP)
    print(f"Document processed into {len(chunks)} chunks")
    
    # Create vector store
    vectorstore = create_vectorstore(chunks, EMBEDDING_MODEL)
    print("Vector store created successfully")
    
    # Create RAG chain
    chain = create_rag_chain(vectorstore, llm, RETRIEVER_K)
    print("RAG chain created successfully")
    
    return chain

## 6. Running the System
Let's run the system on a sample document.

In [9]:
# Build the RAG system
document_url = 'https://media.wizards.com/images/magic/tcg/resources/rules/MagicCompRules_21031101.pdf'
rag_chain = build_rag_system(document_url)

Document processed into 925 chunks
Vector store created successfully
RAG chain created successfully


## 7. Testing with Sample Queries
Let's test our RAG system with some sample queries.

In [10]:
def query_rag(chain, question: str):
    """
    Query the RAG system and display results
    
    Args:
        chain: RAG chain
        question (str): Question to ask
    """
    result = chain.invoke(question)
    
    print(f"\n{'='*50}")
    print(f"Query: {question}")
    print(f"Answer: {result['result']}")
    print("\nSources:")
    for i, doc in enumerate(result['source_documents'][:2], 1):
        print(f"\nSource {i}:")
        print(f"Content: {doc.page_content[:150]}...")
    print(f"{'='*50}\n")
    
    return result

In [11]:
# Test some queries
test_queries = [
    "What is the purpose of the rules in the document?",
    "Can you explain the concept of 'combat' in simple terms?",
    "What are the main sections of the document?",
    "How does one win a game of Magic: The Gathering?",
    "What is the role of the 'stack' in gameplay?"
]

for query in test_queries:
    query_rag(rag_chain, query)


Query: What is the purpose of the rules in the document?
Answer: The rules in this document are for people who know how to play Magic already, but want to know all the little details. (Introduction) They're like the ultimate guide to the game, so you usually only need them for special situations or when you're playing in a big competition. (Introduction) They also help figure out how cards work together.

Sources:

Source 1:
Content: This document includes a series of numbered rules followed by a glossary. Many of the numbered rules are 
divided into subrules, and each separate rul...

Source 2:
Content: rules. 
 
In response to play issues and to keep these rules as current as possible, changes may have been made to 
this document since its publicatio...


Query: Can you explain the concept of 'combat' in simple terms?
Answer: Okay, imagine your toys are having a pretend fight! "Combat" is like that pretend fight. It's a special part of the game where your creatures (your toys) can a